In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import tensorflow as tf

import numpy as np
import libspn as spn

from deepsm.graphspn.spn_model import SpnModel, mod_compute_graph_up
from deepsm.graphspn.tbm.spn_template import TemplateSpn, NodeTemplateSpn, EdgeRelationTemplateSpn
from deepsm.graphspn.tbm.spn_instance import NodeTemplateInstanceSpn
from deepsm.graphspn.tbm.template import ThreeRelTemplate, SingleRelTemplate, SingleTemplate, RelTemplate
from deepsm.graphspn.tbm.graph_builder import build_graph
from deepsm.util import CategoryManager

# Load a trained view template SPN

In [34]:
spn_path = "/home/zkytony/Documents/thesis/experiments/deep-semantic-mapping/deepsm/experiments/results/graphspn/EdgeRelationTemplateExperiment/models/ThreeRelTemplate_10_Freiburg-Saarbrucken.spn"

In [35]:
sess = tf.Session()

In [100]:
params = {
    'num_decomps': 1,
    'num_subsets': 3,
    'num_mixtures': 2,
    'num_input_mixtures': 2,

    # spn_learning
    'additive_smoothing': 30,
}

In [118]:
template_spn = EdgeRelationTemplateSpn(RelTemplate, **params, seed=100)
#template_spn.load(spn_path, sess)
SpnModel.print_structure(template_spn._root, sess)
print(template_spn._conc_inputs)
print(template_spn._view_dist_input)

Node_28 (Sum)
    Input(Products1_1/Product1, None) (Product)
    Input(Products1_1/Product2, None) (Product)
    Input(Products1_1/Product3, None) (Product)
    Input(Products1_1/Product4, None) (Product)
Products1_1/Product1 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_3, [0]) (Concat)
Products1_1/Product2 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_3, [1]) (Concat)
Products1_1/Product3 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_3, [2]) (Concat)
Products1_1/Product4 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_3, [3]) (Concat)
Conc_EdgeRelationTemplateSpn_0_1_3 (Concat)
    Input(View_EdgeRelationTemplateSpn_0_1_3, None) (IVs)
Conc_EdgeRelationTemplateSpn_0_1_3
View_EdgeRelationTemplateSpn_0_1_3


In [119]:
print(template_spn._conc_inputs)
print(template_spn._view_dist_input)

Conc_EdgeRelationTemplateSpn_0_1_3
View_EdgeRelationTemplateSpn_0_1_3


In [120]:
template_spn.expand()

ValueError: num_vars must be a positive integer

In [121]:
spn.display_spn_graph(template_spn._root, skip_params=False)

In [122]:
template_spn._num_subsets

3

In [123]:
template_spn.generate_random_weights()
template_spn.init_weights_ops()
template_spn.init_learning_ops()
template_spn.initialize_weights(sess)

Generating weight initialization Ops...
Initializing learning Ops...
Initializing weights...


In [124]:
template_spn.template

deepsm.graphspn.tbm.template.RelTemplate

In [125]:
SpnModel.print_structure(template_spn._root, sess)

Node_28 (Sum)
    Input(Products1_1/Product1, None) (Product)
    Input(Products1_1/Product2, None) (Product)
    Input(Products1_1/Product3, None) (Product)
    Input(Products1_1/Product4, None) (Product)
Products1_1/Product1 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_3, [0]) (Concat)
Products1_1/Product2 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_3, [1]) (Concat)
Products1_1/Product3 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_3, [2]) (Concat)
Products1_1/Product4 (Product)
    Input(Conc_EdgeRelationTemplateSpn_0_1_3, [3]) (Concat)
Conc_EdgeRelationTemplateSpn_0_1_3 (Concat)
    Input(View_EdgeRelationTemplateSpn_0_1_3, None) (IVs)


In [126]:
template_spn._root.weights

Input(Weights_4/Node_28_Weights, None)

In [129]:
SpnModel.print_weights(template_spn._root, sess)

Node_28 (Sum)
[[ 0.24780256  0.25517696  0.24561906  0.25140151]] [(1, 4)]


In [130]:
template_spn.evaluate(sess, np.array([1]))

array([[-1.365798]], dtype=float32)

In [131]:
template_spn._conc_inputs.set_inputs()
print(template_spn._conc_inputs.inputs)
print(template_spn._num_nodes)
print(template_spn._conc_inputs)
print(template_spn._view_dist_input)

()
0
Conc_EdgeRelationTemplateSpn_0_1_3
View_EdgeRelationTemplateSpn_0_1_3


In [132]:
catg_inputs = spn.IVs(num_vars=9, num_vals=CategoryManager.NUM_CATEGORIES, name="Catg")
view_inputs = spn.IVs(num_vars=3, num_vals=4, name="View")
conc = spn.Concat(catg_inputs, view_inputs)
copied_tspn_root = mod_compute_graph_up(template_spn._root,
                                        TemplateSpn.dup_fun_up,
                                        tmpl_num_vars=[3, 1],
                                        tmpl_num_vals=[CategoryManager.NUM_CATEGORIES, 4],
                                        graph_num_vars=[catg_inputs.num_vars, view_inputs.num_vars],
                                        conc=conc,
                                        labels=[[5,6,7],[0]])

In [113]:
SpnModel.print_weights(copied_tspn_root, sess)

Sum_14 (Sum)
[[ 1.]] [(1, 1)]


In [114]:
SpnModel.print_structure(copied_tspn_root, sess)

Sum_14 (Sum)
    Input(PermProducts_4, None) (PermProducts)
PermProducts_4 (PermProducts)
    Input(Concat_12, [10, 11, 12, 13]) (Concat)
Concat_12 (Concat)
    Input(Catg_12, None) (IVs)
    Input(View_12, None) (IVs)


In [115]:
copied_tspn_root.is_valid()

True